In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [2]:
df_DUS = pd.read_csv('stations/station_6335050_river_data.csv')
df_KOL = pd.read_csv('stations_cologne_fixed/station_6335060_river_data.csv')
df_DUS.date = pd.to_datetime(df_DUS.date, format='%Y-%m-%d')
df_KOL.date = pd.to_datetime(df_KOL.date, format='%Y-%m-%d')

In [3]:
df_DUS['discharge_Day-1'] = df_DUS.discharge.shift(1)
df_DUS['discharge_Day-2'] = df_DUS.discharge.shift(2)
df_DUS['discharge_Day-3'] = df_DUS.discharge.shift(3)
df_DUS['MAVG_Day3'] =  df_DUS['discharge_Day-3'].rolling(window=3).mean()
df_DUS['MAVG_week'] =  df_DUS['discharge_Day-3'].rolling(window=7).mean()
df_DUS_dNull = df_DUS.iloc[9:]


In [11]:
df_KOL['discharge_Day-1'] = df_KOL.discharge.shift(1)
df_KOL['discharge_Day-2'] = df_KOL.discharge.shift(2)
df_KOL['discharge_Day-3'] = df_KOL.discharge.shift(3)
df_KOL['MAVG_Day3'] =  df_KOL['discharge_Day-3'].rolling(window=3).mean()
df_KOL['MAVG_week'] =  df_KOL['discharge_Day-3'].rolling(window=7).mean()
df_KOL_dNull = df_KOL.iloc[9:]

In [6]:
df_DUS_dNull[pd.isnull(df_DUS_dNull).any(axis=1)]

,date,station_no,discharge,water_level,year,month,delta1,delta2,delta3,discharge_Day-1,discharge_Day-2,discharge_Day-3,MAVG_Day3,MAVG_week


In [89]:
df_KOL_dNull[pd.isnull(df_KOL_dNull).any(axis=1)]

,Unnamed: 0,date,station_no,discharge,water_level,year,month,delta1,delta2,delta3,discharge_Day-1,discharge_Day-2,discharge_Day-3,MAVG_Day3,MAVG_Day7,MAVG_week


In [14]:
#df_DUS_dNull = df_DUS.dropna()
#df_KOL_dNull = df_KOL.dropna()

In [5]:
X1_DUS = np.array(df_DUS_dNull['discharge_Day-3']).reshape(-1,1)
X2_DUS = np.array(df_DUS_dNull['MAVG_Day3']).reshape(-1,1)
X3_DUS = np.array(df_DUS_dNull['MAVG_week']).reshape(-1,1)
y0_DUS = np.array(df_DUS_dNull.water_level).reshape(-1,1)
y1_DUS = np.array(df_DUS_dNull.delta1).reshape(-1,1)
y2_DUS = np.array(df_DUS_dNull.delta2).reshape(-1,1)
y3_DUS = np.array(df_DUS_dNull.delta3).reshape(-1,1)
y_DUS = (y1_DUS,y2_DUS,y3_DUS)

In [16]:
X1_KOL = np.array(df_KOL_dNull['discharge_Day-3']).reshape(-1,1)
X2_KOL = np.array(df_DUS_dNull['MAVG_Day3']).reshape(-1,1)
X3_KOL = np.array(df_KOL_dNull['MAVG_week']).reshape(-1,1)
y0_KOL = np.array(df_KOL_dNull.water_level).reshape(-1,1)
y1_KOL = np.array(df_KOL_dNull.delta1).reshape(-1,1)
y2_KOL = np.array(df_KOL_dNull.delta2).reshape(-1,1)
y3_KOL = np.array(df_KOL_dNull.delta3).reshape(-1,1)
y_KOL = (y1_KOL,y2_KOL,y3_KOL)

In [11]:
def performLR(X,y):
    r2_sum = 0
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, test_size= 0.4)
        min_max_scaler = preprocessing.MinMaxScaler()
        X_train_scaled = min_max_scaler.fit_transform(X_train)
        regressor = LinearRegression()  
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        r2_sum += r2
    r2_a= r2_sum/10    
    return r2_a

In [10]:
r2_sum = 0
for i in range(3):
    r2 = performLR(X1_DUS,y_DUS[i])
    r2_sum += r2
    print('R square of y'+str(i+1), r2)  
print("R2_score:", r2_sum/3)

R square of y1 0.07374520501790158
R square of y2 0.07957763111210407
R square of y3 0.06480463274697773
R2_score: 0.07270915629232778


In [11]:
r2_sum = 0
for i in range(3):
    r2 = performLR(X2_DUS,y_DUS[i])
    r2_sum += r2
    print('R square of y'+str(i+1), r2)  
print("R2_score:", r2_sum/3)

R square of y1 0.06548303392738485
R square of y2 0.08706570216564745
R square of y3 0.09438336910043547
R2_score: 0.08231070173115591


In [12]:
r2_sum = 0
for i in range(3):
    r2 = performLR(X3_DUS,y_DUS[i])
    r2_sum += r2
    print('R square of y'+str(i+1), r2)  
print("R2_score:", r2_sum/3)

R square of y1 0.04483242584761123
R square of y2 0.07111508732788083
R square of y3 0.08221738394955497
R2_score: 0.06605496570834901


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X1_DUS, y0_DUS, train_size=0.6, test_size= 0.4)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
regressor = LinearRegression()  
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R2_score:", r2)

R2_score: 0.7219801873342928


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X2_DUS, y0_DUS, train_size=0.6, test_size= 0.4)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
regressor = LinearRegression()  
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R2_score:", r2)

R2_score: 0.6744162955050694


In [75]:
def performSVR(X,y):
    r2_sum = 0
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, test_size= 0.4)
        clf = SVR(kernel='rbf', C=100, epsilon=0.1,tol = 0.01)
    #min_max_scaler = preprocessing.MinMaxScaler()
    #X_train_scaled = min_max_scaler.fit_transform(X_train)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        r2_sum += r2
    r2_a= r2_sum/10      
    return  r2_a

In [96]:
r2_sum = 0
for i in range(3):
    r2 = performSVR(X3_DUS,y_DUS[i])
    r2_sum += r2
print("R2_score:", r2_sum/3)   

C:\Users\Marimo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Marimo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Marimo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Marimo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.2

C:\Users\Marimo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Marimo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Marimo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Marimo\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.2

R2_score: -0.1019076600990091


In [7]:
df_2pred = pd.read_csv('to_predict.csv')
df_river = pd.read_csv('river_data_cologne_fixed.csv')
df_2pred.date = pd.to_datetime(df_2pred.date, format='%Y-%m-%d')
df_river.date = pd.to_datetime(df_river.date, format='%Y-%m-%d')

In [8]:
df_2pred['discharge_Day3'] = pd.Series()
df_2pred['discharge_MAVG3'] = pd.Series()
df_2pred['delta'] = pd.Series()

In [35]:
def get_discharge(station_no,date):
    date_prev = date - pd.Timedelta(days=3)
    discharge_value = df_river[(df_river.date == date_prev)&(df_river.station_no == station_no)].discharge.values
    return discharge_value

In [9]:
def get_discharge_MAVG(station_no,date):
    date_prev = date - pd.Timedelta(days=3)
    if station_no == 6335050:
        discharge_value = df_DUS[(df_DUS.date == date_prev)&(df_DUS.station_no == station_no)].MAVG_Day3.values
    else:
        discharge_value = df_KOL[(df_KOL.date == date_prev)&(df_KOL.station_no == station_no)].MAVG_Day3.values
    return discharge_value


In [42]:
for i in df_2pred.index:
        station_no = df_2pred.loc[i, 'station_no']
        date = df_2pred.loc[i, 'date']
        df_2pred.loc[i, 'discharge_Day3'] = get_discharge(station_no, date)

In [17]:
for i in df_2pred.index:
        station_no = df_2pred.loc[i, 'station_no']
        date = df_2pred.loc[i, 'date']
        df_2pred.loc[i, 'discharge_MAVG3'] = get_discharge_MAVG(station_no, date)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X2_DUS, y1_DUS, train_size=0.6, test_size= 0.4)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
regressor = LinearRegression()  
model1_DUS = regressor.fit(X_train, y_train)
#y_pred = regressor.predict(X_test)  

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X2_KOL, y1_KOL, train_size=0.6, test_size= 0.4)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
regressor = LinearRegression()  
model1_KOL = regressor.fit(X_train, y_train)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X2_DUS, y2_DUS, train_size=0.6, test_size= 0.4)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
regressor = LinearRegression()  
model2_DUS = regressor.fit(X_train, y_train)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X2_KOL, y2_KOL, train_size=0.6, test_size= 0.4)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
regressor = LinearRegression()  
model2_KOL = regressor.fit(X_train, y_train)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X2_DUS, y3_DUS, train_size=0.6, test_size= 0.4)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
regressor = LinearRegression()  
model3_DUS = regressor.fit(X_train, y_train)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X2_KOL, y3_KOL, train_size=0.6, test_size= 0.4)
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
regressor = LinearRegression()  
model3_KOL = regressor.fit(X_train, y_train)

In [38]:
for i in df_2pred.index:
    if i%3 == 0:
        if  df_2pred.loc[i, 'station_no'] == 6335050:
              df_2pred.loc[i, 'delta'] = model1_DUS.predict(df_2pred.loc[i, 'discharge_MAVG3'].reshape(-1,1))[0] 
        else: 
              df_2pred.loc[i, 'delta'] = model1_KOL.predict(df_2pred.loc[i, 'discharge_MAVG3'].reshape(-1,1))[0]  
    elif i%3 == 1:  
        if  df_2pred.loc[i, 'station_no'] == 6335050:
              df_2pred.loc[i, 'delta'] = model2_DUS.predict(df_2pred.loc[i, 'discharge_MAVG3'].reshape(-1,1))[0]  
        else: 
              df_2pred.loc[i, 'delta'] = model2_KOL.predict(df_2pred.loc[i, 'discharge_MAVG3'].reshape(-1,1))[0]  
    elif i%3 == 2:
        if  df_2pred.loc[i, 'station_no'] == 6335050:
              df_2pred.loc[i, 'delta'] = model3_DUS.predict(df_2pred.loc[i, 'discharge_MAVG3'].reshape(-1,1))[0]  
        else: 
              df_2pred.loc[i, 'delta'] = model3_KOL.predict(df_2pred.loc[i, 'discharge_MAVG3'].reshape(-1,1))[0]  
    else:
        print('sth wrong')

In [39]:
df_2pred

,station_no,date,discharge_Day3,discharge_MAVG3,delta
0,6335050,1965-04-17,NaN,2530.000000,1.896835
1,6335050,1965-04-18,NaN,2453.333333,2.778863
2,6335050,1965-04-19,NaN,2393.333333,2.697517
3,6335050,1965-08-01,NaN,3366.666667,7.460646
4,6335050,1965-08-02,NaN,3186.666667,11.934046
...,...,...,...,...,...
985,6335060,2012-08-07,NaN,1622.077333,-8.150047
986,6335060,2012-08-08,NaN,1569.220000,-12.878688
987,6335060,2012-11-20,NaN,2597.466000,2.385028
988,6335060,2012-11-21,NaN,2687.400333,5.919018


In [40]:
df_2pred

,station_no,date,discharge_Day3,discharge_MAVG3,delta
0,6335050,1965-04-17,NaN,2530.000000,1.896835
1,6335050,1965-04-18,NaN,2453.333333,2.778863
2,6335050,1965-04-19,NaN,2393.333333,2.697517
3,6335050,1965-08-01,NaN,3366.666667,7.460646
4,6335050,1965-08-02,NaN,3186.666667,11.934046
...,...,...,...,...,...
985,6335060,2012-08-07,NaN,1622.077333,-8.150047
986,6335060,2012-08-08,NaN,1569.220000,-12.878688
987,6335060,2012-11-20,NaN,2597.466000,2.385028
988,6335060,2012-11-21,NaN,2687.400333,5.919018


In [41]:
df_final1 = df_2pred[['station_no','date','delta']]

In [47]:
df_2pred.delta.mean()

0.07356158267890266

In [46]:
 df_final1.to_csv('final1.csv', index = False)

In [143]:
def get_discharge_DUS(date):
    date_prev = date - pd.Timedelta(days=3)
    discharge = df_DUS[df_DUS.date == date_prev]['discharge']
    return discharge
    
def get_discharge_KOL(date):
    date_prev = date - pd.Timedelta(days=3)
    discharge = df_KOL[df_KOL.date == date_prev]['discharge']
    return discharge

In [ ]:
for (i, row) in df_2pred.iterrows():
    
    if row['station_no'] == 6335050:
        row['discharge_Day3']= get_discharge_DUS(row['date'])
        print(get_discharge_DUS(row['date']))
    elif row['station_no'] == 6335060:
        row['discharge_Day3']= get_discharge_KOL(row['date'])
        print(get_discharge_DUS(row['date']))
    else:
        print('sth wrong')  
    